In [54]:
import json
import pandas as pd
import altair as alt
from langdetect import detect
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
file_paths = [
    '20231012_230826_commit_sharings.json',
    '20231012_232232_hn_sharings.json',
    '20231012_233628_pr_sharings.json',
    '20231012_234250_file_sharings.json',
    '20231012_235128_issue_sharings.json',
    '20231012_235320_discussion_sharings.json'
]

all_data = [] 

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)

        # first get embedded data from Sources column
        for source in data.get('Sources', []):
            source_type = source.get('Type', '')
            source_id = source.get('ID', '')
            source_url = source.get('URL', '')
            author = source.get('Author', '')
            title = source.get('Title', '')
            created_at = source.get('CreatedAt', '')

            # then get embedded data from ChatgptSharing column
            for chatgpt_sharing in source.get('ChatgptSharing', []):
                chatgpt_url = chatgpt_sharing.get('URL', '')
                status = chatgpt_sharing.get('Status', '')
                date_of_conversation = chatgpt_sharing.get('DateOfConversation', '')
                date_of_access = chatgpt_sharing.get('DateOfAccess', '')
                model_used = chatgpt_sharing.get('Model', '')
                num_prompts = chatgpt_sharing.get('NumberOfPrompts', 0)
                tokens_of_prompts = chatgpt_sharing.get('TokensOfPrompts', 0)
                tokens_of_answers = chatgpt_sharing.get('TokensOfAnswers', 0)

                # lastly, organize the conversation information
                for conversation in chatgpt_sharing.get('Conversations', []):
                    entry = {
                        "SourceType": source_type,
                        "SourceID": source_id,
                        "SourceURL": source_url,
                        "Author": author,
                        "Title": title,
                        "CreatedAt": created_at,
                        "ChatGPT_URL": chatgpt_url,
                        "Status": status,
                        "DateOfConversation": date_of_conversation,
                        "DateOfAccess": date_of_access,
                        "ModelUsed": model_used,
                        "NumPrompts": num_prompts,
                        "TokensOfPrompts": tokens_of_prompts,
                        "TokensOfAnswers": tokens_of_answers,
                        "Prompt": conversation.get("Prompt", ""),
                        "Answer": conversation.get("Answer", ""),
                        "ListOfCode": conversation.get("ListOfCode", [])
                    }
                    all_data.append(entry)

df = pd.DataFrame(all_data)

In [3]:
df.head()

,SourceType,SourceID,SourceURL,Author,Title,CreatedAt,ChatGPT_URL,Status,DateOfConversation,DateOfAccess,ModelUsed,NumPrompts,TokensOfPrompts,TokensOfAnswers,Prompt,Answer,ListOfCode
0,commit,,https://github.com/grnpin/textbox/commit/fa335...,grnpin,,,https://chat.openai.com/share/4bad57dd-9636-4b...,200,"September 20, 2023",2023-10-12 23:53:27.348018,Default (GPT-3.5),1,330,556,このコードだと、スマートフォンでキーボードを表示したときにbuttonがキーボードの下に隠れ...,ChatGPTキーボードが表示されたときにボタンをキーボードの上に移動させるには、CSSでキ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1,commit,,https://github.com/grnpin/textbox/commit/0dfc4...,grnpin,,,https://chat.openai.com/share/1fd7ffc2-9264-45...,200,"September 9, 2023",2023-10-12 23:53:27.833077,Default,1,51,160,このコードなんですが、Placeholder上でちゃんと改行するコードにできますか。\n\n...,ChatGPTはい、改行を含むPlaceholder文字列を設定するコードを作成することがで...,[]
2,commit,,https://github.com/grnpin/textbox/commit/811d0...,grnpin,,,https://chat.openai.com/share/99d2d02f-7e3b-43...,200,"September 7, 2023",2023-10-12 23:53:28.345040,Default,1,916,115,refactor this code.\n\n<!DOCTYPE html>\n<html>...,ChatGPTHere's a refactored version of your HTM...,[]
3,commit,,https://github.com/bh679/MindfulAI2.0/commit/e...,bh679,,,https://chat.openai.com/share/90ebe55e-bd60-47...,200,"August 8, 2023",2023-10-12 23:53:29.830292,Advanced Data Analysis,6,5774,1898,server.js\n// Required libraries\nimport cors ...,ChatGPTYou've shared multiple pieces of server...,[]
4,commit,,https://github.com/bh679/MindfulAI2.0/commit/e...,bh679,,,https://chat.openai.com/share/90ebe55e-bd60-47...,200,"August 8, 2023",2023-10-12 23:53:29.830292,Advanced Data Analysis,6,5774,1898,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",ChatGPTIt looks like you're having issues with...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [4]:
df.tail()

,SourceType,SourceID,SourceURL,Author,Title,CreatedAt,ChatGPT_URL,Status,DateOfConversation,DateOfAccess,ModelUsed,NumPrompts,TokensOfPrompts,TokensOfAnswers,Prompt,Answer,ListOfCode
27088,discussion,,https://github.com/ZoeLeBlanc/is578-intro-dh/d...,ZoeLeBlanc,AI & CLI Assignment,2023-08-31T16:33:05Z,https://chat.openai.com/share/cfe2a21a-29e0-47...,200,"September 8, 2023",2023-10-12 23:53:17.501679,Default (GPT-3.5),6,156,1830,received this error code when verifying the st...,ChatGPTI apologize for the confusion. It seems...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
27089,discussion,,https://github.com/ZoeLeBlanc/is578-intro-dh/d...,ZoeLeBlanc,AI & CLI Assignment,2023-08-31T16:33:05Z,https://chat.openai.com/share/cfe2a21a-29e0-47...,200,"September 8, 2023",2023-10-12 23:53:17.501679,Default (GPT-3.5),6,156,1830,now create a how to use the command line to cr...,ChatGPTTo create a text file with the content ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
27090,discussion,,https://github.com/ZoeLeBlanc/is578-intro-dh/d...,ZoeLeBlanc,AI & CLI Assignment,2023-08-31T16:33:05Z,https://chat.openai.com/share/cfe2a21a-29e0-47...,200,"September 8, 2023",2023-10-12 23:53:17.501679,Default (GPT-3.5),6,156,1830,the echo command is not working,ChatGPTI apologize for the confusion. If the e...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
27091,discussion,,https://github.com/ZoeLeBlanc/is578-intro-dh/d...,ZoeLeBlanc,AI & CLI Assignment,2023-08-31T16:33:05Z,https://chat.openai.com/share/cfe2a21a-29e0-47...,200,"September 8, 2023",2023-10-12 23:53:17.501679,Default (GPT-3.5),6,156,1830,the echo command is still not working on windo...,ChatGPTI apologize for the inconvenience. If t...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
27092,discussion,,https://github.com/orgs/netket/discussions/1482,MrAAafa99,Exploring the Plotting of Pair Correlation Fun...,2023-05-25T15:11:27Z,https://chat.openai.com/share/caaa9e43-860a-46...,200,"August 16, 2023",2023-10-12 23:53:19.322053,Default (GPT-3.5),2,36,616,how do I assign error bars to the entries of a...,ChatGPTAssigning error bars to histogram entri...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27093 entries, 0 to 27092
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SourceType          27093 non-null  object
 1   SourceID            27093 non-null  object
 2   SourceURL           27093 non-null  object
 3   Author              27093 non-null  object
 4   Title               27093 non-null  object
 5   CreatedAt           27093 non-null  object
 6   ChatGPT_URL         27093 non-null  object
 7   Status              27093 non-null  int64 
 8   DateOfConversation  27093 non-null  object
 9   DateOfAccess        27093 non-null  object
 10  ModelUsed           27093 non-null  object
 11  NumPrompts          27093 non-null  int64 
 12  TokensOfPrompts     27093 non-null  int64 
 13  TokensOfAnswers     27093 non-null  int64 
 14  Prompt              27093 non-null  object
 15  Answer              27093 non-null  object
 16  ListOfCode          27

In [6]:
# noticed duplicate prompt/answer pairs, so remove these
df = df.drop_duplicates(subset = ["Prompt", "Answer"])

Time for some EDA

In [7]:
df.head()

,SourceType,SourceID,SourceURL,Author,Title,CreatedAt,ChatGPT_URL,Status,DateOfConversation,DateOfAccess,ModelUsed,NumPrompts,TokensOfPrompts,TokensOfAnswers,Prompt,Answer,ListOfCode
0,commit,,https://github.com/grnpin/textbox/commit/fa335...,grnpin,,,https://chat.openai.com/share/4bad57dd-9636-4b...,200,"September 20, 2023",2023-10-12 23:53:27.348018,Default (GPT-3.5),1,330,556,このコードだと、スマートフォンでキーボードを表示したときにbuttonがキーボードの下に隠れ...,ChatGPTキーボードが表示されたときにボタンをキーボードの上に移動させるには、CSSでキ...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."
1,commit,,https://github.com/grnpin/textbox/commit/0dfc4...,grnpin,,,https://chat.openai.com/share/1fd7ffc2-9264-45...,200,"September 9, 2023",2023-10-12 23:53:27.833077,Default,1,51,160,このコードなんですが、Placeholder上でちゃんと改行するコードにできますか。\n\n...,ChatGPTはい、改行を含むPlaceholder文字列を設定するコードを作成することがで...,[]
2,commit,,https://github.com/grnpin/textbox/commit/811d0...,grnpin,,,https://chat.openai.com/share/99d2d02f-7e3b-43...,200,"September 7, 2023",2023-10-12 23:53:28.345040,Default,1,916,115,refactor this code.\n\n<!DOCTYPE html>\n<html>...,ChatGPTHere's a refactored version of your HTM...,[]
3,commit,,https://github.com/bh679/MindfulAI2.0/commit/e...,bh679,,,https://chat.openai.com/share/90ebe55e-bd60-47...,200,"August 8, 2023",2023-10-12 23:53:29.830292,Advanced Data Analysis,6,5774,1898,server.js\n// Required libraries\nimport cors ...,ChatGPTYou've shared multiple pieces of server...,[]
4,commit,,https://github.com/bh679/MindfulAI2.0/commit/e...,bh679,,,https://chat.openai.com/share/90ebe55e-bd60-47...,200,"August 8, 2023",2023-10-12 23:53:29.830292,Advanced Data Analysis,6,5774,1898,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",ChatGPTIt looks like you're having issues with...,"[{'ReplaceString': '[CODE_BLOCK_0]', 'Type': '..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18163 entries, 0 to 27092
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SourceType          18163 non-null  object
 1   SourceID            18163 non-null  object
 2   SourceURL           18163 non-null  object
 3   Author              18163 non-null  object
 4   Title               18163 non-null  object
 5   CreatedAt           18163 non-null  object
 6   ChatGPT_URL         18163 non-null  object
 7   Status              18163 non-null  int64 
 8   DateOfConversation  18163 non-null  object
 9   DateOfAccess        18163 non-null  object
 10  ModelUsed           18163 non-null  object
 11  NumPrompts          18163 non-null  int64 
 12  TokensOfPrompts     18163 non-null  int64 
 13  TokensOfAnswers     18163 non-null  int64 
 14  Prompt              18163 non-null  object
 15  Answer              18163 non-null  object
 16  ListOfCode          18163 n

Let's see how some of the data is distributed

In [9]:
df['Author'].value_counts().head(10)

Author
bennyp85              776
balo215               573
tisztamo              497
caseywschmid          456
i-a-n                 289
ToonTalk              273
Schäffer Krisztián    255
cimonik               254
mmasias               246
koriym                219
Name: count, dtype: int64

In [10]:
df['Title'].value_counts().head(10)

Title
                                                                                  14515
Automation & Testing                                                                166
What happened in this GPT-3 conversation?                                           101
[안승지] 리액트는 프레임 워크인가 라이브러리인가?(리액트로 개발한 프로그램에서 리액트를 대체할 수 있는가?)                        87
What Is Happening with ChatGPT?                                                      75
按照requirement.txt中的numpy版本不能使用import fiftyone                                        68
B"H Trying to run from node.js                                                       63
ChatGPT: Fear Litany                                                                 58
I Tried to Convince Chat-GPT I Was an Advanced AI from 100 Years in the Future       56
データ収集と管理の仕組みの構築                                                                      50
Name: count, dtype: int64

In [11]:
df['NumPrompts'].value_counts()

NumPrompts
1     1416
4      701
3      684
5      655
2      622
      ... 
52      51
51      50
50      49
49      48
48      47
Name: count, Length: 78, dtype: int64

In [12]:
df['ModelUsed'].value_counts()

ModelUsed
Default (GPT-3.5)         6888
GPT-4                     6526
Advanced Data Analysis    2041
Plugins                   2010
Default                    466
Web Browsing               221
Model: Default              11
Name: count, dtype: int64

The prompt counts and model usage seems interesting, let's plot these to see the actual distributions

In [13]:
# don't make the same mistake of using the entire dataset and subset it first
df_prompts = df[['NumPrompts']]

alt.data_transformers.disable_max_rows()

alt.Chart(df_prompts).mark_bar().encode(
    x = alt.X('NumPrompts', bin = alt.Bin(maxbins = 20), title = 'Number of Prompts'),
    y = alt.Y('count()', title = 'Count')
)

alt.Chart(...)

Most conversations are limited to 100 prompts at most, but there are some interesting outliers that may be worth diving into more. Let's group by model

In [14]:
df_model = df[['ModelUsed']]

alt.Chart(df_model).mark_bar().encode(
    x = alt.X('count()', title = 'Count'),
    y = alt.Y('ModelUsed:N', title = 'Model', sort = 'x')
)

alt.Chart(...)

I wonder if there's a relationship between the number of prompts and the model used? This might be difficult to figure out since there are clearly two dominant models, but it might be worth exploring. Additionally, it might be worth figuring out if the higher tier models are actually better and compare the answer to prompts depending on the model. Building on this further, we can evaluate the 'quality' of responses depending on the model.

##### Research Question: How do GPT answers vary based on the model used? Is there specific language or vocabulary that differs between each model? Do the answer lengths differ?

Let's subset the data to only focus on the model, prompts, and answers

In [15]:
df_rq1 = df[['ModelUsed', 'Prompt', 'Answer']]

df_rq1.head()

,ModelUsed,Prompt,Answer
0,Default (GPT-3.5),このコードだと、スマートフォンでキーボードを表示したときにbuttonがキーボードの下に隠れ...,ChatGPTキーボードが表示されたときにボタンをキーボードの上に移動させるには、CSSでキ...
1,Default,このコードなんですが、Placeholder上でちゃんと改行するコードにできますか。\n\n...,ChatGPTはい、改行を含むPlaceholder文字列を設定するコードを作成することがで...
2,Default,refactor this code.\n\n<!DOCTYPE html>\n<html>...,ChatGPTHere's a refactored version of your HTM...
3,Advanced Data Analysis,server.js\n// Required libraries\nimport cors ...,ChatGPTYou've shared multiple pieces of server...
4,Advanced Data Analysis,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",ChatGPTIt looks like you're having issues with...


First, let's combine all the default models into one category

In [16]:
df_rq1.loc[:, 'ModelUsed'] = df_rq1['ModelUsed'].replace(
    ['Model: Default', 'Default'], 'Default (GPT-3.5)'
)

df_rq1.head()

,ModelUsed,Prompt,Answer
0,Default (GPT-3.5),このコードだと、スマートフォンでキーボードを表示したときにbuttonがキーボードの下に隠れ...,ChatGPTキーボードが表示されたときにボタンをキーボードの上に移動させるには、CSSでキ...
1,Default (GPT-3.5),このコードなんですが、Placeholder上でちゃんと改行するコードにできますか。\n\n...,ChatGPTはい、改行を含むPlaceholder文字列を設定するコードを作成することがで...
2,Default (GPT-3.5),refactor this code.\n\n<!DOCTYPE html>\n<html>...,ChatGPTHere's a refactored version of your HTM...
3,Advanced Data Analysis,server.js\n// Required libraries\nimport cors ...,ChatGPTYou've shared multiple pieces of server...
4,Advanced Data Analysis,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",ChatGPTIt looks like you're having issues with...


For interpretability, we will also remove non-english prompts/answers. Additionally, we will remove 'ChatGPT' at the start of each answer as this will skew results if we care about word counts/frequencies.

In [17]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# filter on prompt and answer columns, also remove ChatGPT from the start of Answer
df_rq1_filtered = df_rq1[df_rq1['Prompt'].apply(is_english) & df_rq1['Answer'].apply(is_english)]
df_rq1_filtered.loc[:, 'Answer'] = df_rq1_filtered['Answer'].str.replace(r'^ChatGPT', '', regex  = True)

df_rq1_filtered.head()

,ModelUsed,Prompt,Answer
2,Default (GPT-3.5),refactor this code.\n\n<!DOCTYPE html>\n<html>...,Here's a refactored version of your HTML and J...
3,Advanced Data Analysis,server.js\n// Required libraries\nimport cors ...,You've shared multiple pieces of server code t...
4,Advanced Data Analysis,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",It looks like you're having issues with your s...
5,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,"From the logs and the code you provided, there..."
6,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,The logs indicate two main issues:The module P...


In [18]:
df_rq1_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14314 entries, 2 to 27092
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ModelUsed  14314 non-null  object
 1   Prompt     14314 non-null  object
 2   Answer     14314 non-null  object
dtypes: object(3)
memory usage: 447.3+ KB


Let's re-plot the model counts now that they are grouped properly

In [21]:
df_model = df_rq1_filtered[['ModelUsed']]

alt.Chart(df_model).mark_bar().encode(
    x = alt.X('count()', title = 'Count'),
    y = alt.Y('ModelUsed:N', title = None, sort = 'x')
)

alt.Chart(...)

Let's take a high-level look at models and answers provided, specifically on the average word counts by model. To do this, we will need to perform some extra cleaning, removing punctuation from the text.

In [22]:
def clean_answer(answer):

    # keep contractions good
    answer = re.sub(r"(?<=\w)'(?=\w)", "", answer)
    # take out punctuation, add space
    answer = re.sub(r"[.,;!?:]", " ", answer)
    # deal with any additional symbols, characters, etc.
    answer = re.sub(r"[^\w\s]", "", answer)
    answer = answer.lower()

    return answer

df_rq1_filtered.loc[:, 'Answer'] = df_rq1_filtered['Answer'].apply(clean_answer)

df_rq1_filtered.head()

,ModelUsed,Prompt,Answer
2,Default (GPT-3.5),refactor this code.\n\n<!DOCTYPE html>\n<html>...,heres a refactored version of your html and ja...
3,Advanced Data Analysis,server.js\n// Required libraries\nimport cors ...,youve shared multiple pieces of server code th...
4,Advanced Data Analysis,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",it looks like youre having issues with your se...
5,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,from the logs and the code you provided there...
6,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,the logs indicate two main issues the module p...


Now that the answer column should be mostly clean, we can start doing some proper analysis. Let's start with word counts, a simple metric to calculate.

In [23]:
df_rq1_clean = df_rq1_filtered.copy()
df_rq1_clean.loc[:, 'WordCount'] = df_rq1_clean['Answer'].apply(lambda x: len(x.split()))

df_rq1_clean.head()

,ModelUsed,Prompt,Answer,WordCount
2,Default (GPT-3.5),refactor this code.\n\n<!DOCTYPE html>\n<html>...,heres a refactored version of your html and ja...,85
3,Advanced Data Analysis,server.js\n// Required libraries\nimport cors ...,youve shared multiple pieces of server code th...,514
4,Advanced Data Analysis,"package.json\n{\n ""name"": ""mindfulai"",\n ""ve...",it looks like youre having issues with your se...,343
5,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,from the logs and the code you provided there...,266
6,Advanced Data Analysis,\n/home/bitnami/.pm2/logs/Mindful-AI-out-0.log...,the logs indicate two main issues the module p...,260


In [25]:
summary_stats = df_rq1_clean.groupby('ModelUsed')['WordCount'].agg(
    mean = 'mean',
    median = 'median',
    min = 'min',
    max = 'max',
    std = 'std'
).reset_index()

summary_stats

,ModelUsed,mean,median,min,max,std
0,Advanced Data Analysis,200.347966,164.0,1,1785,163.157819
1,Default (GPT-3.5),166.418981,133.0,1,2478,136.595871
2,GPT-4,161.968727,140.0,1,785,121.575601
3,Plugins,176.986979,154.0,1,1290,122.273996
4,Web Browsing,224.562874,182.0,23,1039,169.818704


We can see that GPT-4 responses have a lot less variance and overall, have a lower word count. The default model, 3.5, has a large range of values, but web browsing and advanced data anlysis, on average, have larger word counts for responses.

Let's extend this slightly to evaluating the frequnecy of words within responses between the models

In [40]:
model_answer_df = df_rq1_clean[['ModelUsed', 'Answer']]

model_answer_df.head()

,ModelUsed,Answer
2,Default (GPT-3.5),heres a refactored version of your html and ja...
3,Advanced Data Analysis,youve shared multiple pieces of server code th...
4,Advanced Data Analysis,it looks like youre having issues with your se...
5,Advanced Data Analysis,from the logs and the code you provided there...
6,Advanced Data Analysis,the logs indicate two main issues the module p...


In [51]:
model_texts = df_rq1_clean.groupby('ModelUsed')['Answer'].apply(lambda x: ' '.join(x))

# vectorization
vectorizer = TfidfVectorizer(stop_words='english')  # remove common words
tfidf_matrix = vectorizer.fit_transform(model_texts)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=model_texts.index, columns=vectorizer.get_feature_names_out())

# top words by model
for model in tfidf_df.index:
    print(f"Top words in {model}:")
    print(tfidf_df.loc[model].sort_values(ascending=False).head(10))
    print()

Top words in Advanced Data Analysis:
data           0.324559
workingshow    0.288158
like           0.206253
file           0.188038
lets           0.167247
finished       0.165959
function       0.158232
using          0.147376
code           0.139833
use            0.137993
Name: Advanced Data Analysis, dtype: float64

Top words in Default (GPT-3.5):
code        0.278328
using       0.226932
use         0.209781
file        0.178849
data        0.176542
heres       0.173289
apolog      0.168305
function    0.166606
example     0.155783
string      0.140938
Name: Default (GPT-3.5), dtype: float64

Top words in GPT-4:
code        0.229491
function    0.229339
use         0.219401
data        0.203393
using       0.189965
like        0.188600
file        0.154688
need        0.142550
heres       0.138529
new         0.116756
Name: GPT-4, dtype: float64

Top words in Plugins:
like        0.256247
function    0.224113
code        0.198325
students    0.187682
use         0.171104
value   

To get a slightly better understanding of the similarities between models, lets do a simple clustering

In [57]:
num_clusters = 3  # set 3 clusters for now since we only have 5 models

kmeans = KMeans(n_clusters=num_clusters, random_state=542, algorithm='elkan')
clusters = kmeans.fit_predict(tfidf_df)

df_clusters = pd.DataFrame({'Model': tfidf_df.index, 'Cluster': clusters})
df_clusters

,Model,Cluster
0,Advanced Data Analysis,0
1,Default (GPT-3.5),2
2,GPT-4,2
3,Plugins,2
4,Web Browsing,1
